# Tokenize and save data

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os
os.chdir('../../')

In [3]:
from src.numpy_encode import *
from src.utils.file_processing import process_all, process_file
from src.config import *
from src.music_transformer import *

In [4]:
# from fastai.text import *

## Preparing the data

In [5]:
version = 'v19'
data_path = Path('data/midi')
version_path = data_path/version

In [6]:
source_dir = 'piano_duet'
# source_dir = 'midi_encode'
out_path = version_path/source_dir
csv_path = out_path/f'{source_dir}.csv'
version_path.ls()

[PosixPath('data/midi/v19/metadata'),
 PosixPath('data/midi/v19/midi_sources'),
 PosixPath('data/midi/v19/midi_encode'),
 PosixPath('data/midi/v19/piano_duet'),
 PosixPath('data/midi/v19/s2s_encode')]

In [7]:
csv = pd.read_csv(csv_path)

/home/ubuntu/anaconda3/envs/midi/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0,1,2,9,10,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
csv['numpy'] = csv['numpy'].apply(lambda x: x.replace('sf4/', '') if isinstance(x, str) else x)
csv.to_csv(csv_path, index=False)

In [9]:
csv = csv.loc[csv['numpy'].notna()];

In [10]:
def create_databunch(files, data_save_name, path=out_path):
    save_file = path/data_save_name
    if save_file.exists():
        data = load_data(path, data_save_name)
    else:
        save_file.parent.mkdir(exist_ok=True, parents=True)
        vocab = MusicVocab.create()
        processors = [OpenNPFileProcessor(), MusicItemProcessor()]

        data = MusicDataBunch.from_files(files, path, processors=processors)
        data.save(data_save_name)
    return data

In [11]:
def get_files(csv):
    files = csv['numpy']
    flist = [Path(version_path/f) for f in files.values if isinstance(f, str)]
    flist = [f for f in flist if f.exists()]
    return flist

In [12]:
version_path

PosixPath('data/midi/v19')

### Create All Dataset

In [13]:
csv.head()

,section,ht_key,song_url,midi,artist,md5,numpy,ht_offset,title,midi_title,parts,genres,mxl,source,ht_mode,ht_time_signature,ht_bpm
0,chorus,C,https://www.hooktheory.com/theorytab/view/wayn...,midi_sources/hooktheory/pianoroll/w/wayne-shar...,wayne-sharpe,bf1f29e5ff84e3e93e37fb873bfb590e,piano_duet/hooktheory/pianoroll/w/wayne-sharpe...,0.0,yu-gi-oh-theme-song,yu-gi-oh3,"intro,chorus",NaN,NaN,hooktheory,1.0,4.0,128.0
1,intro,C,https://www.hooktheory.com/theorytab/view/wayn...,midi_sources/hooktheory/pianoroll/w/wayne-shar...,wayne-sharpe,055f80ad67f64edb14a85ca8fbfe8c29,piano_duet/hooktheory/pianoroll/w/wayne-sharpe...,0.0,yu-gi-oh-theme-song,yu-gi-oh,"intro,chorus",NaN,NaN,hooktheory,1.0,3.0,85.0
2,chorus,D,https://www.hooktheory.com/theorytab/view/what...,midi_sources/hooktheory/pianoroll/w/what-a-day...,what-a-day,197f96f5d181f6ce1e2c5ab04ac1ff87,piano_duet/hooktheory/pianoroll/w/what-a-day/k...,-5.0,kiefer,kiefer,chorus,Jazz,NaN,hooktheory,6.0,4.0,96.0
3,pre-chorus,D,https://www.hooktheory.com/theorytab/view/whit...,midi_sources/hooktheory/pianoroll/w/whiteflame...,whiteflame,9e7ce13a35f1314423a9a6d5a5287a4a,piano_duet/hooktheory/pianoroll/w/whiteflame/s...,-5.0,senbonzakura,senbonzakura - pre-Pre-Chorus,"verse,pre-chorus,chorus","J-Pop,Pop",NaN,hooktheory,6.0,4.0,152.0
4,verse,D,https://www.hooktheory.com/theorytab/view/whit...,midi_sources/hooktheory/pianoroll/w/whiteflame...,whiteflame,d5aaf79d0989222f1362f9f46c540a27,piano_duet/hooktheory/pianoroll/w/whiteflame/s...,-5.0,senbonzakura,Senbonzakura,"verse,pre-chorus,chorus","J-Pop,Pop",NaN,hooktheory,6.0,4.0,152.0


In [14]:
all_files = get_files(csv); len(all_files)

112169

## Testing

In [15]:
import random
# sample_data = create_databunch(random.sample(all_files, 1000), data_save_name='cached/sample.pkl')

In [16]:
files = random.sample(all_files, 1000)
path = out_path

vocab = MusicVocab.create()
processors = [OpenNPFileProcessor(), MusicItemProcessor()]

data = MusicDataBunch.from_files(files, path, processors=processors, encode_position=True)

item = data.train_dl.dataset.x[0]

type(item)

item.data

a = np.load(files[0])

a

files[0]

In [23]:
np.load('data/midi/v17/sf4/piano_duet/130k_reddit/M/M/Maxime.npy')

array([[71,  4],
       [71,  7],
       [67,  4],
       [67,  7],
       ...,
       [69,  1],
       [64,  4],
       [61,  4],
       [57,  1]])

In [24]:
item.position

array([ 0,  0,  0,  0, ..., 96, 96, 96, 96])

## End TEsting

In [26]:
all_data = create_databunch(all_files, data_save_name='cached/all.pkl')

In [27]:
all_data.one_batch()

(tensor([[ 61, 145,   8,  ..., 153,  63, 153],
         [ 61, 145,   8,  ..., 153,  63, 153],
         [ 61, 145,   8,  ..., 153,  63, 153],
         ...,
         [ 61, 145,   8,  ..., 153,  63, 153],
         [ 61, 145,   8,  ..., 153,  63, 153],
         [ 61, 145,   8,  ..., 153,  63, 153]]),
 tensor([[145,   8, 145,  ...,  63, 153,  60],
         [145,   8, 145,  ...,  63, 153,  60],
         [145,   8, 145,  ...,  63, 153,  60],
         ...,
         [145,   8, 145,  ...,  63, 153,  60],
         [145,   8, 145,  ...,  63, 153,  60],
         [145,   8, 145,  ...,  63, 153,  60]]))

### Create sample

In [28]:
import random
sample_data = create_databunch(random.sample(all_files, 1000), data_save_name='cached/sample.pkl')

Note: we are reusing all_vocab for the following datasets

### Create Hooktheory Dataset

In [29]:
hook_csv = csv.loc[csv.source.isin(['hooktheory'])]
hook_files = get_files(hook_csv); len(hook_files)

19404

In [30]:
hook_data = create_databunch(hook_files, 'cached/hook.pkl')

### Create Hooktheory C Dataset

In [31]:
hook_csv = csv.loc[csv.source.isin(['hooktheory_c'])]
hook_files = get_files(hook_csv); len(hook_files)

19562

In [32]:
hook_data = create_databunch(hook_files, 'cached/hook_c.pkl')

## Low quality

In [33]:
lq_csv = csv.loc[csv.source.isin(['reddit', 'classical_piano', 'ecomp', 'midiworld', 'freemidi', 'lmd', 'cprato', 'wikifonia', 'classical_archives'])]
lq_files = get_files(lq_csv); len(lq_files)

64006

In [34]:
lq_data = create_databunch(lq_files, 'cached/lq.pkl')

## All High quality

In [35]:
hq_csv = csv.loc[csv.source.isin(['hooktheory', 'musescore'])]
hq_files = get_files(hq_csv); len(hq_files)

28601

In [36]:
hq_data = create_databunch(hq_files, 'cached/hq.pkl')

### Testing

In [37]:
data = all_data

In [38]:
data.train_ds.x

MusicItemList (100953 items)
xxbos xxpad xxsep d1 n55 d1 xxsep d1 n54 d1 xxsep d1 n45 d1 xxsep d1 n53 d1 xxsep d1 n52 d1 xxsep d1 n45 d1 xxsep d1 n53 d1 xxsep d1 n52 d1 xxsep d1 n45 d3 xxsep d3 n55 d1 xxsep d1 n54 d1 xxsep d1 n55 d1 xxsep d1 n61 d1 xxsep d1 n59 d16 n55 d16 n52 d16 xxsep d16 n60 d8 n57 d8 n55 d8 n52 d8 xxsep d8 n59 d8 n55 d8 n52 d8 xxsep d8 n50 d24 n46 d24 n43 d24 xxsep d24 n52 d4 n48 d4 n45 d4 n43 d4 xxsep d4 n54 d4 n50 d4 n47 d4 n45 d4 xxsep d4 n55 d16 n51 d16 n48 d16 xxsep d16 n62 d8 n60 d8 n56 d8 n53 d8 xxsep d8 n55 d8 n51 d8 n48 d8 xxsep d8 n58 d16 n54 d16 n51 d16,xxbos xxpad n65 d1 xxsep d1 n67 d14 xxsep d14 n67 d1 xxsep d1 n65 d1 xxsep d1 n67 d1 xxsep d1 n65 d6 xxsep d6 n64 d24 xxsep d28 n60 d4 xxsep d4 n67 d1 xxsep d1 n65 d1 xxsep d1 n67 d1 xxsep d1 n68 d9 xxsep d9 n67 d1 xxsep d1 n68 d1 xxsep d1 n67 d24 xxsep d24 n67 d2 xxsep d2 n70 d10 xxsep d10 n69 d1 xxsep d1 n70 d1 xxsep d1 n69 d1 xxsep d1 n67 d5 xxsep d5 n68 d1 xxsep d1 n67 d1 xxsep d1 n65 d8 xxsep d8 n67 

In [ ]:
# train_ids_file = out_path/'tmp/all/train_ids.npy'
# all_ids = np.load(train_ids_file)
# id_cat = np.concatenate(all_ids); id_cat.shape
# ax = tuple(range(len(id_cat.shape)-1))
# max_vocab = id_cat.max(axis=ax)
# max_vocab = (max_vocab+1).tolist(); max_vocab

In [ ]:
npenc2stream(data.train_ds.x[10]).show()